In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import os
import re
import pickle
import sklearn
import sys
import string

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier,NearestNeighbors
from sklearn.svm import LinearSVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../src')
sys.path.append(src_dir)

In [3]:
%aimport data.movielens_20m_imdb
%aimport features.movielens_imdb
%aimport helpers.files,helpers.labels

In [4]:
from data.movielens_20m_imdb import load_into_dataframe
from features.movielens_imdb import filter_rare_tags
from helpers.labels import truncate_labels

In [5]:
INTERIM_DATA_ROOT = "../../data/interim/movielens-ml20m-imdb/"
ML_ROOT = "/media/felipe/SAMSUNG/movielens/ml-20m/"
IMDB_ROOT = "/media/felipe/SAMSUNG/imdb/"

PATH_TO_MOVIES = ML_ROOT + "/movies.csv"
PATH_TO_TAG_ASSIGNMENTS = ML_ROOT + "/tags.csv"
# slightly modified version, without the top 15 lines
PATH_TO_MOVIE_PLOTS = IMDB_ROOT+"/plot.list.tail"

# CONFIGS
MAX_NB_WORDS = 3000
NB_NEIGHBOURS = 10
DISTANCE_METRIC='cosine'
MIN_TAG_DOC_COUNT = 2 # like delicious t-140

In [6]:
if os.path.isfile(INTERIM_DATA_ROOT+"docs_df.p"):
    docs_df = pickle.load( open( INTERIM_DATA_ROOT+"docs_df.p", "rb" ))
else:
    docs_df = load_into_dataframe(PATH_TO_MOVIES,PATH_TO_TAG_ASSIGNMENTS,PATH_TO_MOVIE_PLOTS)
    pickle.dump(docs_df,open(INTERIM_DATA_ROOT+"docs_df.p","wb"))

In [7]:
docs_df.head(10)

,movie_id,title,tags,plot
0,1,Toy Story (1995),"family,light,fun,want-to-see-again,classic,kid...",A little boy named Andy loves to be in his ro...
1,2,Jumanji (1995),"jungle,chris-van-allsburg,time,dynamic-cgi-act...",After being trapped in a jungle board game fo...
2,3,Grumpier Old Men (1995),"howard-deutch,sequel,sequel-fever,comedinha-de...",Things don't seem to change much in Wabasha C...
3,4,Waiting to Exhale (1995),"characters,chick-flick,clv,revenge",This story based on the best selling novel by...
4,5,Father of the Bride Part II (1995),"family,childhood-classics,it-thought-it-was-fu...","In this sequel to ""Father of the Bride"", Geor..."
5,6,Heat (1995),"somber,btaege,violent,erlends-dvds,cant-rememb...",Hunters and their prey--Neil and his professi...
6,7,Sabrina (1995),"harrison-ford,relationships,ugly-duckling,dram...","While she was growing up, Sabrina Fairchild s..."
7,8,Tom and Huck (1995),"library-system,authormark-twain,based-on-a-boo...","A mischievous young boy, Tom Sawyer ('Jonatha..."
8,9,Sudden Death (1995),"clv,peter-hyams,action,jeanclaude-van-damme,ca...",Some terrorists kidnap the Vice President of ...
9,10,GoldenEye (1995),"007-series,bond,seen-more-than-once,btaege,fra...",When a deadly satellite weapon system falls i...


In [8]:
data = docs_df['plot'].values
labelsets = docs_df["tags"].map(lambda tagstring: tagstring.split(","))

labelsets=truncate_labels(labelsets,MIN_TAG_DOC_COUNT)

In [11]:
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(labelsets)